### download the last month of the CDS raw forecasts 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser

In [3]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
HOME = pathlib.Path.home()

In [5]:
CWD = pathlib.Path.cwd()

In [6]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [7]:
sys.path.append('../..')

In [8]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [9]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [120, 240, -38, 22.5]}

### provider (always CDS for now)

In [10]:
provider = 'CDS'

### variable name

In [11]:
var_name = 'tprate'

### list of GCMs 

In [12]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC']

### mapping GCM to systems 

In [13]:
from collections import OrderedDict as OD

In [14]:
dict_systems = OD(
                ECMWF=5, # SEAS5, hindcast complete 
                UKMO=600, # note: only for system 14 is hindcast complete
                METEO_FRANCE=8, # note: only for system 7 is hindcast complete
                DWD=21, # GCFS2.1, hindcast complete 
                CMCC=35, # SPSv3.5, hindcast complete 
                NCEP=2,# CFSv2, hindcast complete 
                JMA=2, # CPS2, hindcast complete
                ECCC_GEM_NEMO=1, # GEM-NEMO, hindcast incomplete
                ECCC_CanCM4i=2, # CanCM4i, hindcast incomplete
                )

### lag in months (if need to download older forecasts)

In [15]:
lag = 0 

### get today's date 

In [16]:
date = dateparser.parse('today')

In [17]:
date

datetime.datetime(2021, 11, 17, 9, 39, 45, 824414)

### apply lag 

In [18]:
date = date - relativedelta(months=lag)

In [19]:
print(f"will download the forecasts issued in {date:%B %Y}")

will download the forecasts issued in November 2021


### path to download the realtime forecasts 

In [20]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts')

In [21]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### use dask to parallelize the download of the forecasts 

In [22]:
list_GCMs

['ECMWF', 'UKMO', 'METEO_FRANCE', 'CMCC', 'DWD', 'NCEP', 'JMA', 'ECCC']

In [24]:
downloaded_files = []

for GCM in dict_systems.keys():
    
    if 'ECCC' in GCM:
        
        GCM_API_name = GCM.split('_')[0]
        GCM_path = gcm_path.joinpath(GCM).joinpath(var_name.upper())
        
        print(f"processing ECCC, path should be {str(GCM_path)}")
    
        args = dict(GCM=GCM_API_name,
        varname=var_name,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=GCM_path,
        domain=domains.domains['C3S_download'],
        file_format='netcdf',
        level='surface',
        max_retry=3)
    
    else: 
        
        args = dict(GCM=GCM,
        varname=var_name,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
        domain=domains.domains['C3S_download'],
        file_format='netcdf',
        level='surface',
        max_retry=3)    
    
    downloaded_files.append(delayed(C3S.download)(**args))


processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_GEM_NEMO/TPRATE
processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_CanCM4i/TPRATE


In [25]:
with ProgressBar():
    downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_METEO_FRANCE.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_JMA.netcdf exists already on disk, skipping download and returning path

/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/DWD/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_DWD.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_CMCC.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_NCEP.netcdf exists already on disk, skippin

2021-11-17 09:40:35,552 INFO Welcome to the CDS
2021-11-17 09:40:35,555 INFO Welcome to the CDS
2021-11-17 09:40:35,556 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-11-17 09:40:35,558 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels


[###############################         ] | 77% Completed |  2.1s

2021-11-17 09:40:36,063 INFO Request is completed
2021-11-17 09:40:36,065 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data1/adaptor.mars.external-1637016363.898373-30320-13-de9209f7-2025-47fa-8b44-8cd67fbd9a94.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_CanCM4i/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_ECCC.netcdf (2.2M)
2021-11-17 09:40:36,073 INFO Request is completed
2021-11-17 09:40:36,076 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data1/adaptor.mars.external-1637016363.898373-30320-13-de9209f7-2025-47fa-8b44-8cd67fbd9a94.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_GEM_NEMO/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_ECCC.netcdf (2.2M)


[###############################         ] | 77% Completed |  4.3s

...

[###################################     ] | 88% Completed | 42.9s


 99%|█████████▉| 2.16M/2.18M [00:38<00:00, 150kB/s]

[###################################     ] | 88% Completed | 43.0s


                                                   2021-11-17 09:41:16,942 INFO Download rate 54.6K/s


[###################################     ] | 88% Completed | 43.4s
/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_CanCM4i/TPRATE/ensemble_seas_forecasts_tprate_from_2021_11_ECCC.netcdf downloaded OK

[########################################] | 100% Completed | 43.5s


In [26]:
# for GCM in list_GCMs: 
    
#     f = C3S.download(
#     GCM=GCM,
#     varname=var_name,
#     year=date.year,
#     month=date.month,
#     leadtimes=[1, 2, 3, 4, 5, 6],
#     opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
#     domain=domains.domains['C3S_download'],
#     file_format='netcdf',
#     level='surface',
#     max_retry=3,
#     )   
#     lfiles.append(f)

### check dimensions and return the total number of members in the C3S MME 

In [27]:
import xarray as xr

In [28]:
members = []
for f in downloaded_files[0]: 
    dset = xr.open_dataset(f)
    members.append(dset.dims['number'])
    print(dset.dims)
    if 'system' in dset.dims:
        print(dset['system'])
    

Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 62, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 124, 'time': 5})
Frozen({'longitude': 57, 'latitude': 33, 'number': 78, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([1, 2], dtype=int32)
Coordinates:
  * system   (system) int32 1 2
Attributes:
    long_name:  system
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([1, 2], dtype=int32)
Coordinates:
  * system   (system) int32 1 2
Attributes:
    long_name:  system


In [28]:
import numpy as np 

In [29]:
np.array(members).sum()

476